In [22]:
import polars as pl
import os
import sqlite3

In [23]:
path = "/home/itamarst/devel/bluebikes-importer/bluebike.sqlite" # os.path.abspath("bluebikes.sqlite")
assert os.path.exists(path)
print(path)
conn = sqlite3.connect(path)
db = pl.read_database(query="SELECT id, start_id, end_id, started_at FROM bluebikes", connection=conn, schema_overrides={"start_id": pl.Utf8(), "end_id": pl.Utf8()})

/home/itamarst/devel/bluebikes-importer/bluebike.sqlite


In [24]:
db = db.with_columns(pl.col("started_at").str.to_date("%Y-%m-%d %H:%M:%S%.f"))
db

id,start_id,end_id,started_at
i64,str,str,date
1,"""345""","""235""",2022-09-01
2,"""141""","""329""",2022-09-01
3,"""68""","""381""",2022-09-01
4,"""78""","""381""",2022-09-01
5,"""22""","""113""",2022-09-01
…,…,…,…
1901181312,"""41""","""369""",2021-02-28
1901181313,"""110""","""115""",2021-02-28
1901181314,"""47""","""44""",2021-02-28


In [25]:
stations = pl.read_csv("data/bluebike_stations_geolocated.csv", infer_schema_length=10000)
cambridge_station_ids = stations.filter(pl.col("City") == "Cambridge")
cambridge_station_ids

id,name,latitude,longitude,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str
"""M32036""","""Rindge Avenue - O'Neill Librar…",42.392857,-71.129058,42.392914,-71.129285,1.0,"""rooftop""","""1""","""Yerxa Rd""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02140""","""US""","""Office of Geographic Informati…"
"""177""","""University Park""",42.362648,-71.100061,42.362691,-71.100102,1.0,"""rooftop""","""13""","""Sidney St""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02139""","""US""","""Office of Geographic Informati…"
"""M32065""","""Kennedy-Longfellow School 158 …",42.369553,-71.08579,42.369728,-71.085637,1.0,"""rooftop""","""153""","""Spring St""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02141""","""US""","""City of Cambridge"""
"""M32077""","""Cambridge Crossing at North Fi…",42.371141,-71.076198,42.37125,-71.076335,1.0,"""rooftop""","""125""","""Monsignor O'brien Hwy""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02141""","""US""","""Office of Geographic Informati…"
"""182""","""Museum of Science""",42.36769,-71.071163,42.367704,-71.071183,1.0,"""rooftop""","""1""","""Science Park""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02141""","""US""","""City of Cambridge"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""M32006""","""MIT at Mass Ave / Amherst St""",42.3581,-71.093198,42.358217,-71.093367,1.0,"""rooftop""","""40""","""Massachusetts Ave""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02139""","""US""","""City of Cambridge"""
"""96""","""Cambridge Main Library at Broa…",42.373379,-71.111075,42.373249,-71.11132,1.0,"""rooftop""","""448""","""Broadway""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02138""","""US""","""City of Cambridge"""
"""M32090""","""O'Brien Highway at First Stree…",42.370982,-71.076638,42.371029,-71.07656,1.0,"""nearest_street""",null,"""MA-28""",null,null,"""Cambridge""","""MA""","""Middlesex County""","""02141""","""US""","""TIGER/Line® dataset from the U…"


In [26]:
cambridge_ids = cambridge_station_ids.get_column("id") 
cambridge_rides = db.filter(pl.col("start_id").is_in(cambridge_ids) | (pl.col("end_id").is_in(cambridge_ids)))
cambridge_rides

id,start_id,end_id,started_at
i64,str,str,date
2,"""141""","""329""",2022-09-01
3,"""68""","""381""",2022-09-01
4,"""78""","""381""",2022-09-01
6,"""68""","""471""",2022-09-01
7,"""179""","""189""",2022-09-01
…,…,…,…
1901181307,"""318""","""178""",2021-02-28
1901181309,"""80""","""11""",2021-02-28
1901181310,"""228""","""6""",2021-02-28


In [17]:
print(2019, len(cambridge_rides.filter(pl.col("started_at").dt.year() == 2019).rows()))
print(2018, len(cambridge_rides.filter(pl.col("started_at").dt.year() == 2018).rows()))

2019 1182680
2018 873172


In [28]:
from datetime import date

#print(2018, len(cambridge_rides.filter((pl.col("started_at").dt.date() >= date(2018, 9, 1)) & (pl.col("started_at").dt.date() <= date(2019, 8, 31)) ).rows()))
#print(2022, len(cambridge_rides.filter((pl.col("started_at").dt.date() >= date(2022, 9, 1)) & (pl.col("started_at").dt.date() <= date(2023, 8, 31)) ).rows()))
print("Last 12 months", len(cambridge_rides.filter((pl.col("started_at").dt.date() >= date(2023, 9, 1)) & (pl.col("started_at").dt.date() <= date(2024, 8, 31)) ).rows()))

Last 12 months 2037820


In [21]:
print("Jan-Aug 2023", len(cambridge_rides.filter((pl.col("started_at").dt.date() >= date(2023, 1, 1)) & (pl.col("started_at").dt.date() <= date(2023, 8, 31)) ).rows()))
print("Jan-Aug 2024", len(cambridge_rides.filter((pl.col("started_at").dt.date() >= date(2024, 1, 1)) & (pl.col("started_at").dt.date() <= date(2024, 8, 31)) ).rows()))

Jan-Aug 2023 1148563
Jan-Aug 2024 1384109
